In [1]:
%cd /home/admin_mcn/hungvq

/home/admin_mcn/hungvq


In [2]:
from stable_diffusion.ldm.util import instantiate_from_config
from omegaconf import OmegaConf
import os
import torch
import numpy as np
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Resize, ToPILImage
from torchvision.transforms.functional import to_pil_image

In [4]:
%cd /home/admin_mcn/hungvq/stable_diffusion

/home/admin_mcn/hungvq/stable_diffusion


In [5]:
config_path = '/home/admin_mcn/hungvq/stable_diffusion/logs/2023-06-07T11-32-09_seg_diff_autoencoder/configs/2023-06-07T11-32-09-project.yaml'
ckpt_path = '/home/admin_mcn/hungvq/stable_diffusion/logs/2023-06-07T11-32-09_seg_diff_autoencoder/checkpoints/last.ckpt'

In [6]:
config = OmegaConf.load(config_path)
model = instantiate_from_config(config.model)

making attention of type 'vanilla' with 256 in_channels
Working with z of shape (1, 32, 12, 12) = 4608 dimensions.
making attention of type 'vanilla' with 256 in_channels
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth


In [7]:
ckpt = torch.load(ckpt_path, map_location='cpu')
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [8]:
MASK_DIR = '/home/admin_mcn/thaotlp/data/ISIC/mask'
LATENT_GT_DIR = '/home/admin_mcn/hungvq/data/latent_gt'

In [10]:
for filepath in os.listdir(MASK_DIR):
    fileidx = filepath.split(".")[0]
    trans = Compose([ToTensor(), Resize((384,384))])
    mask = Image.open(os.path.join(MASK_DIR, filepath)).convert(mode='L')

    latent_z = model.encode(trans(mask).unsqueeze(0)).sample()

    latent_gt = latent_z.detach().cpu().numpy()

    np.save(os.path.join(LATENT_GT_DIR, fileidx + '.npy'), latent_gt)

    

In [14]:
latent_z.shape

torch.Size([1, 32, 12, 12])

In [ ]:
latent_output = torch.Tensor(np.load('/home/admin_mcn/thaotlp/data/ISIC/latent_output/ISIC_0010264.npy'))
output = model.decode(latent_output)
to_pil_image(torch.clamp(output[0].squeeze(0),min=0,max=1), mode='RGB').convert('L')

In [ ]:
latent_output = torch.Tensor(np.load('/home/admin_mcn/thaotlp/data/ISIC/latent_output/ISIC_0000002.npy'))
output = model.decode(latent_output)
to_pil_image(torch.clamp(output[0].squeeze(0),min=0,max=1), mode='RGB').convert('L')

In [ ]:
latent_gt = torch.Tensor(np.load('/home/admin_mcn/hungvq/data/latent_gt/ISIC_0000002.npy'))
gt = model.decode(latent_gt)
to_pil_image(torch.clamp(gt[0].squeeze(0),min=0,max=1), mode='L')